In [ ]:
!jupyter nbextension enable --py widgetsnbextension


In [ ]:
#import cv2
import torch
import imageio
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torch
import torch.nn.functional as F

from tqdm.notebook import tqdm
from torch import Tensor
from itertools import compress, cycle
from collections import OrderedDict
from scipy.interpolate import griddata
from IPython.display import Image

from utils.utils import *

from utils.plotting import Plotter
from utils.gl_solver import GLSolver
from utils.parameters_init import ParametersInit
from utils.random_input_field import RandomInputField

import warnings
warnings.filterwarnings('ignore')

EPOCHS = 20_000
LR = 1
# SEED = 1234

# np.random.seed(SEED)
# torch.manual_seed(SEED)
# torch.cuda.manual_seed(SEED)
# torch.cuda.manual_seed_all(SEED)
# torch.set_default_tensor_type(torch.FloatTensor)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:

Nx=128
Ny=128

Lx= 50
Ly= 50
T_end = 1
dt = 0.005

myubatch_size = 8192
myu=16
path = f"main-newarchitecture-N-{Nx}-L-{Lx}-Myu-{myu}-new-approach"
mtlibpath_prefix = path + "_mtl"

input_to_defect_ratio_sqrt = Nx // myu
N_ITERATIONS = int(T_end / dt)
A_norm, A_original, mem_rate, myu_original = compute_A_norm(
    Nx=Nx, 
    Ny=Ny, 
    input_to_defect_ratio=input_to_defect_ratio_sqrt*input_to_defect_ratio_sqrt, 
    mean=5.4, 
    std_deviation=0.8, 
    time_period=25, 
    Lx=Lx, 
    Ly=Ly, 
    dt=dt, 
    T_End=T_end, 
    parallel_runs=1, 
    input_scale=0.75, 
    mem_coef=1, 
    time_period_parameter=50, 
    _mean=5.4, 
    std_deviation_run_computation=1,
    input_myu=None
)

GETTING HANDS DIRTY WITH THE NEURAL NETWORKS

In [ ]:
x = np.linspace(0, Lx, Nx).flatten()[:, None]
y = np.linspace(0, Ly, Ny).flatten()[:, None]
t = np.linspace(0, T_end, N_ITERATIONS).flatten()[:, None]

Exact = A_original.squeeze(0)

X, T, Y = np.meshgrid(x, t, y)

X_star = np.hstack((X.flatten()[:, None], Y.flatten()[:, None], T.flatten()[:, None]))
u_star = Exact.flatten()
u_star = np.hstack([u_star.real[:, None],u_star.imag[:, None]])

$$
\begin{aligned}
\partial_{t} A &= \mu A+\Delta A-|A|^{2} A
\end{aligned}
$$

In [ ]:
class FCN(nn.Module):
    def __init__(self, layers_list, activation_function_list = None):
        nn.Module.__init__(self)

        self._depth = len(layers_list) - 1
        if activation_function_list is None:
            activation_function_list = [torch.nn.Softplus for _ in range(self._depth - 1)]        
        
        seq = []
        for i, activation_function in enumerate(activation_function_list):
            seq.append(('layer_%d' % i, torch.nn.Linear(layers_list[i], layers_list[i+1], dtype = torch.float64)))
            seq.append(('activation_%d' % i, activation_function()))
        
        seq.append(('layer_%d' % (self._depth - 1), torch.nn.Linear(layers_list[-2], layers_list[-1], dtype = torch.float64)))
        
        self._Wtmx = torch.nn.Sequential(OrderedDict(seq))
        self.optimizer = torch.optim.Adam( params = self._Wtmx.parameters(), lr=0.01 )    
        
    def forward(self, x):
        return self._Wtmx(x)
    
class MShuffle(nn.Module):
    def __init__(self, exp_size, n_depth, activation_function_list = None):
        nn.Module.__init__(self)
        
        if activation_function_list is None:
            activation_function_list = [torch.nn.Softplus]*exp_size
        
        seq = []

        for nd in range(n_depth):
            for n, activation_function in enumerate(activation_function_list):
                seq.append(('layer_d_%d_n_%d' % (nd + 1, n + 1), torch.nn.Linear(2**exp_size, 2**(exp_size-1), dtype = torch.float64)))
                seq.append(('activation_d_%d_n_%d' % (nd + 1,n + 1), activation_function()))

        self._Wtmx = torch.nn.Sequential(OrderedDict(seq))
        self.n_depth = n_depth

        mask = np.zeros((exp_size,*[2] * exp_size),dtype = bool)
        for e in range(exp_size):
            exec(f"mask[{e},{':,'*e+'0'}] = True")
        self.mask = torch.tensor(mask.reshape(exp_size,-1),dtype=bool) 

    def forward(self,x):
        Seq = self._Wtmx.children()
        for _ in range(self.n_depth):
            for m in self.mask:
                x_new = x.clone()
                x_new[:,m] = next(Seq)(next(Seq)(x))
                x = x_new
        return x


In [ ]:
from itertools import chain

class ALLINPUTNET(nn.Module):
    def __init__(self):
        super(ALLINPUTNET, self).__init__()
        self.encoder = FCN(layers_list = [3,4,16,64,256])
        self.decoder = FCN(layers_list = [256,64,16,4,2])
        self.shuffler = MShuffle(exp_size = 8, n_depth = 2)
        self.optimizer = torch.optim.Adam(params = chain(
            self.encoder._Wtmx.parameters(),
            self.shuffler._Wtmx.parameters(),
            self.decoder._Wtmx.parameters(),
            ), lr=0.0001)

        
    def forward(self, x):
        E = self.encoder.forward(x)
        S = self.shuffler.forward(E)
        return self.decoder.forward(S)

    def predict(self, x):
        self.eval()
        if type(x) is not torch.Tensor:
            x = torch.tensor(x, dtype = torch.float64).to(device)
        y =  self.forward(x).cpu().detach().numpy()
        return y[:,0] + y[:,1]*1j    

    def rmsef(self, y, y_pred):
        mseloss = torch.sum((y_pred - y)**2, dim = 1)
        return torch.mean(torch.sqrt(mseloss))   

    def msef(self, y, y_pred):
        return torch.mean((y_pred - y)**2)       

    def zero_grad(self, set_to_none: bool = False) -> None:
        self.encoder._Wtmx.zero_grad()
        self.shuffler._Wtmx.zero_grad()
        self.decoder._Wtmx.zero_grad()
        return super().zero_grad(set_to_none)

    def y_proc(y):
        MS = y[:,1] * y[:,0]
        IR = y[:,1] ** 2 +  y[:,0] ** 2
        return torch.vstack((MS,IR)).T
        
    def fastmsebatchtrain(self, x, y, epochs=100, batch_size = 64):
        
        x = torch.tensor(x, dtype = torch.float64).to(device)
        y = torch.tensor(y, dtype = torch.float64).to(device)
        dataloader = DataLoader(dataset = torch.hstack((x,y)), batch_size=batch_size, shuffle=True)
        
        self.optimizer.zero_grad()
        L = []
        
        pbar = tqdm(total=epochs)
        try:
            while True:
                if pbar.n >= epochs:
                    break
                for tmp in dataloader:
                    (tmpx, tmpy, tmpt, tmpu_real, tmpu_img) = tmp.T
                    X = torch.stack((tmpx,tmpy,tmpt)).T
                    U = torch.stack((tmpu_real, tmpu_img)).T

                    y_pred = self.forward(X)
                    loss = self.msef(y_pred,U)
                    L.append(loss.cpu().detach().numpy())
                    loss.backward()
                    self.optimizer.step()
                    self.zero_grad()
                    self.optimizer.zero_grad()
                    # Update the progress bar
                    if pbar.n >= epochs:
                        break
                    pbar.update(1)
                        
        except Exception as e:
            raise Exception(e)
        finally:
            # Close the progress bar
            pbar.close()          
        return L
    
    def fastrmsebatchtrain(self, x, y, epochs=100, batch_size = 64):
        
        x = torch.tensor(x, dtype = torch.float64).to(device)
        y = torch.tensor(y, dtype = torch.float64).to(device)
        dataloader = DataLoader(dataset = torch.hstack((x,y)), batch_size=batch_size, shuffle=True)
        
        self.optimizer.zero_grad()
        L = []
        
        pbar = tqdm(total=epochs)
        try:
            while pbar.n < epochs:
                for tmp in dataloader:
                    (tmpx, tmpy, tmpt, tmpu_real, tmpu_img) = tmp.T
                    X = torch.stack((tmpx,tmpy,tmpt)).T
                    U = torch.stack((tmpu_real, tmpu_img)).T

                    y_pred = self.forward(X)
                    loss = self.rmsef(y_pred,U)
                    L.append(loss.cpu().detach().numpy())
                    loss.backward()
                    self.optimizer.step()
                    self.zero_grad()
                    self.optimizer.zero_grad()
                    # Update the progress bar
                    if pbar.n >= epochs:
                        break
                    pbar.update(1)
                        
        except Exception as e:
            raise Exception(e)
        finally:
            # Close the progress bar
            pbar.close()          
        return L

In [ ]:
class ALLINPUTPINN(ALLINPUTNET):
    def __init__(self,*args,**kwargs):
        super(ALLINPUTPINN, self).__init__(*args,**kwargs)
#         myu = torch.randn(4, 2, dtype=torch.float64).to(device)
#         myu = nn.Parameter(myu)
#         self._Wtmx.register_parameter('myu', myu)
#         self.myuparam = myu
#         myu = transform_and_stack(myu, 4, 200).to(device).clone().requires_grad_(True)
#         self.myu = myu.view(200, 4, 4)
        self.myureset()
    
    def myureset(self):
        myu = torch.randn(mem_rate, Nx//input_to_defect_ratio_sqrt, Ny//input_to_defect_ratio_sqrt, dtype=torch.float64).to(device)
        myu = nn.Parameter(myu)
        self.myuparam = myu
        
    @property
    def myu(self):
        myu = F.interpolate(self.myuparam.unsqueeze(0), scale_factor=input_to_defect_ratio_sqrt, mode='nearest').squeeze()
        return myu
    
    def loaddata_precalculate(self,x):
        myuloss = MYULOSS(*tuple(x.T),self)
        myuloss.calculate_f_withoutmyu()
        self.myuloss = myuloss
    
    def fmsef(self, myu):
        return self.myuloss.fmse(myu)
    
    def fmse(self):
        return self.fmsef(self.myu)
    
    def myutrain(self, epochs=100, lr = 0.01):
        
        myuoptimizer = torch.optim.Adam( params = [self.myuparam], lr=lr ) 
        myuoptimizer.zero_grad()

        for _ in tqdm(range(epochs)):
            self.fmse().backward(retain_graph=True)
            myuoptimizer.step()
            myuoptimizer.zero_grad()
        
        FMSE = self.myuloss.FMSE
        self.myuloss.clear()
        return FMSE

In [ ]:
class MYULOSS:
  def __init__(self, x, y, t, net, verbose = 0):
      self.msef = nn.MSELoss()
      self.FMSE = []
      self.x = x
      self.y = y
      self.t = t
      self.net = net
        
  def plot(self, title= 'MYU training'):
    plt.plot(self.FMSE)
    plt.yscale('log')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('FMSE')
    self.clear()
    
  def clear(self):
    self.FMSE = []
    
  def fmse(self, myu):
    f_loss = torch.mean(torch.abs(self.net_f(myu)) ** 2)
    self.FMSE.append(f_loss.cpu().detach().numpy())
    return f_loss
  
  def net_f(self, myu, verbose = 0):
        return self.f_withoutmyu - myu*self.u 
    
  def calculate_f_withoutmyu(self):
        x,y,t = self.x, self.y, self.t
        
        u, u_t,u_xx,u_yy = MYULOSS.pref(x,y,t,net)
        
        self.u = u.cpu().detach()
            
        f_withoutmyu = u_t - (u_xx + u_yy) + torch.pow(torch.abs(u), 2)*u #- myu*u
        self.f_withoutmyu = f_withoutmyu.cpu().detach()
        free_memory(u_t, u_xx, u_yy, u, f_withoutmyu)

    
  def f_withoutmyu(x,y,t,ru,iu):
        (ru_t, ru_x, ru_y) = torch.autograd.grad(ru, (t, x, y), grad_outputs=torch.ones_like(ru), create_graph=True, retain_graph=True)
        (iu_t, iu_x, iu_y) = torch.autograd.grad(iu, (t, x, y), grad_outputs=torch.ones_like(iu), create_graph=True, retain_graph=True)

        (ru_xx,) = torch.autograd.grad(ru_x, (x), grad_outputs=torch.ones_like(ru_x), create_graph=True)
        (iu_xx,) = torch.autograd.grad(iu_x, (x), grad_outputs=torch.ones_like(iu_x), create_graph=True)

        (ru_yy,) = torch.autograd.grad(ru_y, (y), grad_outputs=torch.ones_like(ru_y), create_graph=True)
        (iu_yy,) = torch.autograd.grad(iu_y, (y), grad_outputs=torch.ones_like(iu_y), create_graph=True)

        u =( ru + iu * 1j)
        u_t = (ru_t + iu_t * 1j)
        u_xx =( ru_xx + iu_xx *1j)
        u_yy = (ru_yy + iu_yy *1j)
        
        return u, u_t,u_xx,u_yy
    
  def pref(x,y,t, net, batch_size = myubatch_size):
    dataloader = DataLoader(dataset = X_star, batch_size = batch_size, shuffle=False)
    cache = {
        'u':[],
        'u_t':[],
        'u_xx':[],
        'u_yy':[],
    }
    for tmp in tqdm(dataloader):
        x,y,t = torch.tensor(tmp.T, dtype = torch.float64, requires_grad=True).to(device)
        ru,iu = net.forward(torch.stack((x,y,t)).T).T
        u, u_t,u_xx,u_yy = MYULOSS.f_withoutmyu(x,y,t,ru,iu)
        cache['u'].append(u.cpu().detach())
        cache['u_t'].append(u_t.cpu().detach())
        cache['u_xx'].append(u_xx.cpu().detach())
        cache['u_yy'].append(u_yy.cpu().detach())
        
    return  torch.cat(cache['u']).view(mem_rate, Nx, Ny), \
            torch.cat(cache['u_t']).view(mem_rate, Nx, Ny),\
            torch.cat(cache['u_xx']).view(mem_rate, Nx, Ny),\
            torch.cat(cache['u_yy']).view(mem_rate, Nx, Ny)

      
def free_memory(*variables):
    del variables
    torch.cuda.empty_cache()

In [ ]:
torch.manual_seed(0)
device = 'cpu'
net = ALLINPUTPINN().to(device)
u_star_proc = ALLINPUTPINN.y_proc(torch.tensor(u_star, dtype = torch.float64).to(device))
L1 = net.fastrmsebatchtrain(x = X_star, y = u_star_proc, epochs = 3000)

In [ ]:
plt.plot(L1)
plt.yscale('log')
plt.xlabel('epochs')
plt.ylabel('Custom Loss')
plt.title('Training of the AllInputNet \n lr=0.01')
plt.savefig(f'{mtlibpath_prefix}_allinputnet001.png')

In [ ]:
net.optimizer.param_groups[0]['lr'] = 0.001
#L2 = net.fastbatchtrain(x = X_star, y_proc = u_star_proc, epochs = 100000)

## calculating myus

In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)
net.loaddata_precalculate(X_star)
net.myureset()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)
net.myureset()

i = 0
for lr in [10, 3, 1, 0.3]:
    L = net.myutrain(lr=lr, epochs=10)
    l = net.fmse().cpu().detach().numpy()
    L += [l]
    net.myuloss.clear()
 
    plt.plot(range(i,i+len(L)), L, label=f'lr={lr}')
    i+=len(L)-1

plt.yscale('log')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('FMSE')
plt.title('MYU Training')
plt.tight_layout()
plt.savefig(f'{mtlibpath_prefix}_myutraining.png')
plt.show()

## Visualizing and saving plot gifs

In [ ]:
def process_frame(index, u_im_real, o_im_real, phase,  title):
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    
    # Display the u_pred image
    im1 = axs[0].imshow(u_im_real[index])
    axs[0].set_title(title + " (u_pred) without normalization" + f" - Frame: {index}")
    axs[0].title.set_position([.5, 1.05])
    
    # Display the original image
    im2 = axs[1].imshow(o_im_real[index])
    axs[1].set_title(title + " (original) without normalization" + f" - Frame: {index}")
    axs[1].title.set_position([.5, 1.05])
    
    # Display the difference image
    im3 = axs[2].imshow(np.abs(u_im_real[index] - o_im_real[index]))
    axs[2].set_title(title + " (difference) without normalization" + f" - Frame: {index}")
    axs[2].title.set_position([.5, 1.05])
    
    # Add colorbars
    fig.colorbar(im1, ax=axs[0], fraction=0.046, pad=0.04)
    fig.colorbar(im2, ax=axs[1], fraction=0.046, pad=0.04)
    fig.colorbar(im3, ax=axs[2], fraction=0.046, pad=0.04)
    
    # Draw the figure and store the image
    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    plt.close(fig)
    
    return image

In [ ]:
def process_subframe(fig, ax, m, title):
    
    im = ax.imshow(m)
    ax.title.set_position([.5, 1.05])
    im = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    ax.set_title(title)
    
def process_frame(M, titles):
    figsize = (M.shape[0], M.shape[1])
    fig, axs = plt.subplots(*figsize, figsize=(18, 6))
    
    # Draw
    for i in range(figsize[0]):
        for j in range(figsize[1]):
            ax = axs[i][j]
            process_subframe(fig,axs[i][j], M[i][j],titles[i][j])
            
    # Draw the figure and store the image
    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    plt.close(fig)
    
    return image   

def create_video(M,titles, videotitle, save=True, fps=30):
    
    
    num_cores = multiprocessing.cpu_count()
#    images = Parallel(n_jobs=num_cores)(delayed(process_frame)((m,titles) for m in tqdm(M)))
    images = [process_frame(m,titles) for m in M]
    if save:
        images_to_video(images, videotitle, fps=30)
        print("Video successfully saved at", videotitle)
    
    return None

def create_video(M,titles, videotitle, save=True, fps=30):
    
    
    images = [process_frame(m,titles) for m in M]
    #Parallel(n_jobs=num_cores)(delayed(process_frame)(m, titles) for m in tqdm(M))
    if save:
        # Convert images to 8-bit color for video
        images = [cv2.cvtColor(img, cv2.COLOR_RGB2BGR) for img in images]
        height, width, _ = images[0].shape
        video = cv2.VideoWriter(videotitle, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
        for img in images:
            video.write(img)
        video.release()
        print("Video successfully saved at", videotitle)
    
    return None

In [ ]:
from __future__ import annotations
import cv2
import torch
import imageio
import numpy as np
import multiprocessing
import matplotlib.pyplot as plt

from tqdm import tqdm
from typing import List, Optional
from joblib import Parallel, delayed
from torch.nn.parameter import Parameter
from IPython.display import display, Image



In [ ]:
module_sq = lambda a: np.real(a)**2 + np.imag(a)**2
phase = lambda a: np.arcsin(np.real(a)/np.sqrt(np.real(a)**2+np.imag(a)**2))
real_imag = lambda a: np.real(a)*np.imag(a)
real = lambda a: np.real(a)
imag = lambda a: np.imag(a)

funlist_name = ["module_sq", "phase","real_imag","real","imag"]
funlist = [module_sq, phase, real_imag, real, imag]

myu = myu_original[0]
myupred = net.myu.cpu().detach().numpy()

A_pred = net.predict(X_star).reshape(A_original[0].shape)

In [ ]:
ATenzor = np.array([np.stack([np.stack([fun(A) for fun in funlist]),
                    np.stack([fun(Ap) for fun in funlist])]) for A, Ap in zip(A_original[0], A_pred)])

In [ ]:
MTenzor = np.array([np.array([m,mp]) for m, mp in zip(myu,myupred)])

In [ ]:
AMTenzor = np.concatenate((MTenzor[:, :, np.newaxis, :, :], ATenzor), axis=2)

In [ ]:
create_video(AMTenzor, titles=[['myu'] + funlist_name]*2, videotitle = path+'.mp4')